In [1]:
import gc
# Clear all variables to free up memory
for name in dir():
    if not name.startswith('_') and name not in ['gc']:
        del globals()[name]

gc.collect()
print("All variables cleared and garbage collected")
import pandas as pd
import os
import sys
import dask.dataframe as dd

os.chdir('/shared/share_scp/coresignal/gitrepo_facebook')

if '/shared/share_scp/coresignal/gitrepo_facebook' in sys.modules:
    for module_name in list(sys.modules.keys()):
        if module_name.startswith('university_name_matcher'):
            del sys.modules[module_name]

from university_name_matcher import university_name_matcher
umatcher = university_name_matcher()

os.chdir('/shared/share_scp/coresignal')

All variables cleared and garbage collected


In [49]:
del globals()['all_experience']



# Changelog 
### Changes before 10/6
1. Data now spans all people how finished college form 1995 to 2012 

### Changes after meeting 10/6

1. The 'founder_inc' variables are re-created with much higher values. The issue was that they were being matched as case-sensitive (i.e., 'inc' matched, but not 'Inc' or 'INC') and they had not included 'company' or 'co'
2. The 'owner' variable had been cleaned to remove all definitions such as 'program owner' that were more jobs, but the coowner variable still included them, these have been removed.


# 1. Convert the raw files into an analysis file
These are the files created from Python CSVs into Pandas

### 1.1 Read the files on education experience

In [32]:
universities_adopted_facebook = umatcher.load_university_data() 
x = universities_adopted_facebook[universities_adopted_facebook['instnm'].duplicated()].instnm

print("Number of duplicate university IDs:", universities_adopted_facebook[universities_adopted_facebook['instnm'].isin(x)].shape[0])


print("Number of NA in name", universities_adopted_facebook['instnm'].isna().sum())

Loaded 6528 universities from the dataset
Kept 6300 unique universities after removing duplicates
Number of duplicate university IDs: 0
Number of NA in name 0


In [ ]:
import glob
education_files = sorted(glob.glob('processed_data2/coresignal_member_education_*linkedin*.pkl'))
print(f'Found {len(education_files)} education files')
coresignal_member_education_all = pd.concat(
    [pd.read_pickle(f) for f in education_files],
    ignore_index=True
)

coresignal_member_education_all['member_id'] = coresignal_member_education_all['member_id'].astype(int)
coresignal_member_education_all = coresignal_member_education_all.drop_duplicates()
member_ids = coresignal_member_education_all['member_id'].astype(int).unique()
print(f"Number of unique member IDs: {len(member_ids)}")

In [ ]:
#Scratch
print("Checking for duplicates in the combined DataFrame")
dups0= coresignal_member_education_all.duplicated(['id']).sum()
total0 = coresignal_member_education_all.shape[0]
coresignal_member_education_all = coresignal_member_education_all[~coresignal_member_education_all.title.str.contains("university of phoenix|devry university", case=False, na=False)]
dups1 = coresignal_member_education_all.duplicated(['id']).sum()
total1 = coresignal_member_education_all.shape[0]

pct0 = dups0 / total0 * 100
pct1 = dups1 / total1 * 100
print(f"Total duplicates found: {dups0:,} ({pct0:.2f}%). Dropped to {dups1:,} ({pct1:.2f}%) after filtering for 'university of phoenix|devry university'")


In [ ]:

import numpy as np
from Levenshtein import distance as levenshtein_distance

# Function to safely calculate Levenshtein distance
def safe_levenshtein(str1, str2):
    if pd.isna(str1) or pd.isna(str2):
        return np.nan
    return levenshtein_distance(str(str1).lower(), str(str2).lower())

# Calculate Levenshtein distance using .loc to avoid SettingWithCopyWarning
coresignal_member_education_all.loc[:, 'levenshtein_distance'] = coresignal_member_education_all.apply(
    lambda row: safe_levenshtein(row['title'], row['instnm']), 
    axis=1
)

# Display the results
print("Levenshtein distances between title and university name:")


In [ ]:
coresignal_member_education_all.loc[:, 'is_duplicated'] = coresignal_member_education_all.duplicated(['id'])

pd.concat([coresignal_member_education_all[coresignal_member_education_all['is_duplicated']].sample(10),
              coresignal_member_education_all[~coresignal_member_education_all['is_duplicated']].sample(10)],
              ignore_index=True)[['id', 'member_id', 'title', 'instnm', 'levenshtein_distance', 'is_duplicated']]



In [ ]:
import matplotlib.pyplot as plt

# Create subplots for comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot histogram for non-duplicated records
non_duplicated_distances = coresignal_member_education_all[~coresignal_member_education_all['is_duplicated']]['levenshtein_distance'].dropna()
ax1.hist(non_duplicated_distances, bins=50, alpha=0.7, color='blue', edgecolor='black')
ax1.set_title('Levenshtein Distance Distribution\n(Non-Duplicated Records)')
ax1.set_xlabel('Levenshtein Distance')
ax1.set_ylabel('Frequency')
ax1.grid(True, alpha=0.3)

# Plot histogram for duplicated records
duplicated_distances = coresignal_member_education_all[coresignal_member_education_all['is_duplicated']]['levenshtein_distance'].dropna()
ax2.hist(duplicated_distances, bins=50, alpha=0.7, color='red', edgecolor='black')
ax2.set_title('Levenshtein Distance Distribution\n(Duplicated Records)')
ax2.set_xlabel('Levenshtein Distance')
ax2.set_ylabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"Non-duplicated records - Mean distance: {non_duplicated_distances.mean():.2f}, Median: {non_duplicated_distances.median():.2f}")
print(f"Duplicated records - Mean distance: {duplicated_distances.mean():.2f}, Median: {duplicated_distances.median():.2f}")
print(f"Non-duplicated count: {len(non_duplicated_distances):,}")
print(f"Duplicated count: {len(duplicated_distances):,}")

In [ ]:
coresignal_member_education_all = coresignal_member_education_all[~coresignal_member_education_all['is_duplicated']]

In [ ]:
total_rows = coresignal_member_education_all.shape[0]
coresignal_member_education_all = coresignal_member_education_all[~coresignal_member_education_all['school_url'].str.endswith('linkedin.com/edu/school')]
print(f"Total rows before filtering: {total_rows:,}")
print(f"Total rows after filtering:  {coresignal_member_education_all.shape[0]:,}")

In [ ]:
# Filter for bachelor's degrees or undergrad
coresignal_member_education = coresignal_member_education_all[
    coresignal_member_education_all['subtitle'].str.lower().str.contains(r'bachelor\'?s?|undergrad|\sb\.a\.|b\.s\.', na=False)
]
print(f"Filtered from {len(coresignal_member_education_all):,} to {len(coresignal_member_education):,} records for bachelor's or undergrad degrees.")


print(f"Deleting coresignal_member_education_all object")

del coresignal_member_education_all
gc.collect()

### 1.2 Read the files on employment experience

In [ ]:
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing
import time

def read_pickle_file(filepath):
    return pd.read_pickle(filepath)

# Find all processed pickle files matching the pattern
os.chdir('/shared/share_scp/coresignal') #make sure it is in the right directory

processed_files = sorted(glob.glob('processed_data/coresignal_member_experience_*START*.pkl'))
print(f'Found {len(processed_files):,} files with experience.')

# Use parallel processing with manual progress tracking
max_workers = min(len(processed_files), multiprocessing.cpu_count())
print(f'Using {max_workers} workers for parallel processing.')

dataframes = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_file = {executor.submit(read_pickle_file, file): file 
                     for file in processed_files}
    
    # Process completed tasks and show progress
    for i, future in enumerate(as_completed(future_to_file), 1):
        dataframes.append(future.result())
        
        # Print progress every 10 files or at the end
        if i % 10 == 0 or i == len(processed_files):
            elapsed = time.time() - start_time
            rate = i / elapsed if elapsed > 0 else 0
            remaining = len(processed_files) - i
            eta = remaining / rate if rate > 0 else 0
            
            print(f"Progress: {i:,}/{len(processed_files):,} files ({i/len(processed_files)*100:.1f}%) | "
                  f"Rate: {rate:.1f} files/sec | "
                  f"Time Left: {eta:.0f}s | "
                  f"Elapsed: {elapsed:.0f}s")

print("Concatenating dataframes...")
all_experience = pd.concat(dataframes, ignore_index=True)

print(f"Total rows in all experience data: {len(all_experience):,}")


In [ ]:
# limit to those with education records that will be our focus
all_experience = all_experience[all_experience['member_id'].isin(coresignal_member_education['member_id'])]
print(f"Total rows in all experience data after filtering to undergrad graduates: {len(all_experience):,}")

In [ ]:

import time

start_time = time.time()
print(f"Total rows in all experience data: {len(all_experience):,}")
print("Removing duplicates...")
key_columns = ['member_id','location','company_url','duration','order_in_profile','company_id', 'company_name', 'title', 'date_from', 'date_to']
all_experience = all_experience.drop_duplicates(subset=['id'])
print(f"Total rows in all experience data (after removing duplicates): {len(all_experience):,}")
elapsed_time = time.time() - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)
print(f"Time taken to remove duplicates: {minutes}m {seconds}s")

### 1.3 Testing and validations

In [ ]:
# Plot the number of entries by graduation year


# Create subplots for graduation year and start year
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot graduation year
grad_year_counts = coresignal_member_education.loc[(coresignal_member_education['year_to'] >= 1994) & (coresignal_member_education['year_to'] <= 2011),'year_to'].value_counts().sort_index()
ax1.bar(grad_year_counts.index, grad_year_counts.values, alpha=0.7, color='steelblue')
ax1.set_title('Number of Education Entries by Graduation Year (1994-2011)')
ax1.set_xlabel('Graduation Year')
ax1.set_ylabel('Number of Entries')
ax1.grid(True, alpha=0.3, axis='y')
ax1.tick_params(axis='x', rotation=45)

# Plot start year
start_year_counts = coresignal_member_education.loc[(coresignal_member_education['year_from'] >= 1994) & (coresignal_member_education['year_from'] <= 2011),'year_from'].value_counts().sort_index()
ax2.bar(start_year_counts.index, start_year_counts.values, alpha=0.7, color='orange')
ax2.set_title('Number of Education Entries by Start Year (1994-2011)')
ax2.set_xlabel('Start Year')
ax2.set_ylabel('Number of Entries')
ax2.grid(True, alpha=0.3, axis='y')
ax2.tick_params(axis='x', rotation=45)


In [ ]:
# Get top 50 universities by number of graduates
top_50_universities = coresignal_member_education['instnm'].value_counts().head(50)

# Create the plot
plt.figure(figsize=(15, 10))
bars = plt.barh(range(len(top_50_universities)), top_50_universities.values, color='steelblue', alpha=0.7)

# Customize the plot
plt.yticks(range(len(top_50_universities)), top_50_universities.index, fontsize=8)
plt.xlabel('Number of Graduates')
plt.title('Top 50 Universities by Number of Graduates in Dataset')
plt.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, top_50_universities.values)):
    plt.text(value + max(top_50_universities.values) * 0.01, i, f'{value:,}', 
             va='center', fontsize=7)

# Invert y-axis so highest counts are at top
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Filter data for University of Texas at Austin
ut_austin_data = coresignal_member_education[
    coresignal_member_education['instnm'] == 'The University of Texas at Austin'
]

# Get graduation year counts for UT Austin
ut_austin_grad_years = ut_austin_data['year_to'].value_counts().sort_index()

# Create the plot
plt.figure(figsize=(12, 6))
plt.bar(ut_austin_grad_years.index, ut_austin_grad_years.values, color='orange', alpha=0.7)
plt.title('Number of Graduates by Year - University of Texas at Austin')
plt.xlabel('Graduation Year')
plt.ylabel('Number of Graduates')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for year, count in ut_austin_grad_years.items():
    plt.text(year, count + max(ut_austin_grad_years.values) * 0.01, 
             f'{count:,}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

print(f"Total UT Austin graduates in dataset: {len(ut_austin_data):,}")
print(f"Year range: {ut_austin_grad_years.index.min()} - {ut_austin_grad_years.index.max()}")


### 1.4 Output and save the analysis files

In [ ]:
coresignal_member_education.to_pickle('coresignal_member_education_AnalysisFile_09302025.pkl')


In [ ]:

all_experience.to_pickle('all_experience_AnalysisFile_09302025.pkl')


# 2. Create the Stata analysis files from the complete datasets
Data loading from base files is done and now we add the things we are going to study

## 2.1 Setup and variable creation

### 2.1.1 Load datasets

In [2]:
# Loads data from pickle files instead of re-creating it. 
# Uncomment as necessary

coresignal_member_education = pd.read_pickle('coresignal_member_education_AnalysisFile_09302025.pkl')
all_experience = pd.read_pickle('all_experience_AnalysisFile_09302025.pkl')


### 2.1.2 Create owner variables directly from titles

In [3]:
tot0 = all_experience.shape[0]
all_experience = all_experience[~all_experience.date_from.isnull()]
tot1 = all_experience.shape[0]
print(f"Dropped {tot0 - tot1:,} rows with null date_from ({((tot0 - tot1)/tot0*100):.2f}%)")

Dropped 337,948 rows with null date_from (0.51%)


In [4]:
#all_experience = all_experience.sample(frac=.3)
# This cell takes about 5 minutes to run
print("variable: job_from",flush=True)
all_experience['job_from'] = all_experience['date_from'].str.extract(r'(\d{4})').astype(float)

print("variable: is_founder_only",flush=True)
all_experience['is_founder_only'] = all_experience['title'].str.contains('founder', case=False, na=False) 

print("variable: is_owner_only",flush=True)
all_experience['is_owner_only'] = all_experience['title'].str.contains(r'owner\b', case=False, na=False)

print("variable: is_founder_or_owner",flush=True)
all_experience['is_founder_or_owner'] = all_experience['is_founder_only'] | all_experience['is_owner_only']

print("variable: is_founder_or_owner_with_url, is_founder_or_owner_inc",flush=True)
all_experience['is_founder_or_owner_with_url'] = all_experience['is_founder_or_owner'] & all_experience['company_url'].notna()
all_experience['is_founder_or_owner_inc'] = all_experience['is_founder_or_owner'] & all_experience['company_name'].str.contains(r'\b(inc|corp|corporation|co|incorporated)\b', na=False, case=False)

print("variables: cofounder/coowner titles",flush=True)
all_experience['is_cofounder_or_coowner_title'] = all_experience['title'].str.contains(r'co[-\s]?founder|co[-\s]?owner', case=False, na=False)
all_experience['is_cofounder_only_title'] = all_experience['title'].str.contains(r'co[-\s]?founder', case=False, na=False)
all_experience['is_coowner_only_title'] = all_experience['title'].str.contains(r'co[-\s]?owner', case=False, na=False)



variable: job_from
variable: is_founder_only
variable: is_owner_only
variable: is_founder_or_owner
variable: is_founder_or_owner_with_url, is_founder_or_owner_inc


/tmp/8359077.1.debian.q/ipykernel_259104/2965000107.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_experience['is_founder_or_owner_inc'] = all_experience['is_founder_or_owner'] & all_experience['company_name'].str.contains(r'\b(inc|corp|corporation|co|incorporated)\b', na=False, case=False)


variables: cofounder/coowner titles


In [5]:


owner_columns = [col for col in all_experience.columns if 'owner' in col.lower()]
#all_experience = dd.from_pandas(all_experience, npartitions=6)

all_experience['job_from'] = all_experience['date_from'].str.extract(r'(\d{4})').astype('Int64')
all_experience['long_title'] = all_experience['title'].str.split().str.len() >= 4

print(f"creating mask_non_owners", flush=True)
mask_non_owners =(all_experience['long_title'] | 
        all_experience['title'].str.contains(r'(product|assistant to|business process|program|process) owner', case=False, na=False) |        
        (all_experience['title'] == "Owner Advisor")
)         
owners = all_experience[owner_columns].fillna(False)
owners = owners.mask(cond=mask_non_owners, other=False)
all_experience[owner_columns] = owners
    


creating mask_non_owners


/tmp/8359077.1.debian.q/ipykernel_259104/4222476495.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_experience['title'].str.contains(r'(product|assistant to|business process|program|process) owner', case=False, na=False) |


### 2.1.3 Create flag for franchising

In [6]:
# Ensure company_name is categorical for efficiency
all_experience['company_name'] = all_experience['company_name'].astype('category')

# --- Founders and owners separately ---
founders = (
    all_experience[all_experience['is_founder_only']]
    .groupby('company_name', observed=True)['member_id']
    .nunique()
    .rename('num_unique_founders')
)

owners = (
    all_experience[all_experience['is_owner_only']]
    .groupby('company_name', observed=True)['member_id']
    .nunique()
    .rename('num_unique_owners')
)

# --- Combined (either founder or owner) ---
founder_or_owner = (
    all_experience[all_experience['is_founder_only'] | all_experience['is_owner_only']]
    .groupby('company_name', observed=True)['member_id']
    .nunique()
    .rename('num_unique_founders_or_owners')
)

# --- Merge all together ---
companies_ownership_by_name = (
    pd.concat([founders, owners, founder_or_owner], axis=1)
    .fillna(0)
    .reset_index()
    .sort_values('num_unique_founders_or_owners', ascending=False)
)


In [7]:
companies_ownership_by_name = companies_ownership_by_name[companies_ownership_by_name['company_name'].notna() & (companies_ownership_by_name['company_name'] != '-')]
companies_ownership_by_name = companies_ownership_by_name[~companies_ownership_by_name['company_name'].str.match(r'self[-\s]?employed|^freelance$|^consultant$|^independent consultant$|^business owner$|^private practice$|^Stealth startup$|^stealth|^startup|^entrepreneur$', case=False, na=False)]
companies_ownership_by_name = companies_ownership_by_name[~companies_ownership_by_name['company_name'].str.match(r'^\'?self', case=False, na=False)]
companies_ownership_by_name = companies_ownership_by_name[~(companies_ownership_by_name['company_name'] == ".")]
companies_ownership_by_name = companies_ownership_by_name[~(companies_ownership_by_name['company_name'].isin(['owner','confidential','none']))]
companies_ownership_by_name['share_founders_vs_owners'] = companies_ownership_by_name['num_unique_founders'] / companies_ownership_by_name['num_unique_founders_or_owners'].replace(0, pd.NA)



#There are some odd companies giving 'founder' titles around
mask = (companies_ownership_by_name['share_founders_vs_owners'] > 0.7)
companies_ownership_by_name  = companies_ownership_by_name[~mask]




In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(companies_ownership_by_name.tail(100))


,company_name,num_unique_founders,num_unique_owners,num_unique_founders_or_owners,share_founders_vs_owners
153028,Ackerson Birth Services,0.0,1.0,1,0.0
153059,Acres Lawn Service,0.0,1.0,1,0.0
153058,Acreman Guitars,0.0,1.0,1,0.0
153057,"Acredale Crest Stables, LLC",0.0,1.0,1,0.0
153056,AcreMax Risk Management LLC,0.0,1.0,1,0.0
153055,Acre Insurance,0.0,1.0,1,0.0
153054,Acre,0.0,1.0,1,0.0
153053,Acquired Taste Booking,0.0,1.0,1,0.0
153052,Acquire Realty RE,0.0,1.0,1,0.0
153051,Acqua Villa,0.0,1.0,1,0.0


In [9]:
franchises = companies_ownership_by_name['num_unique_founders_or_owners'] > 15 & (companies_ownership_by_name['share_founders_vs_owners'] < 0.1)

all_experience['franchise_founder_or_owner'] = all_experience['company_name'].isin(companies_ownership_by_name[franchises]['company_name']) & all_experience['is_founder_or_owner']
all_experience['franchise_owner'] = all_experience['company_name'].isin(companies_ownership_by_name[franchises]['company_name']) & all_experience['is_owner_only']

### 2.1.4 Develop different definitions of co-owner / co-founder.

In [10]:
all_experience['company_url'] = all_experience['company_url'].astype('category')
found_own = all_experience[all_experience['is_founder_or_owner']] 
companies_ownership_by_url = found_own.groupby('company_url', as_index=False, observed = True).agg({
    'is_founder_only': 'sum',
    'is_owner_only': 'sum'
}).sort_values(['is_founder_only', 'is_owner_only'], ascending=False)

companies_ownership_by_url = companies_ownership_by_url[companies_ownership_by_url['company_url'].notna()]


companies_ownership_by_url['total_founders_owners'] = companies_ownership_by_url['is_founder_only'] + companies_ownership_by_url['is_owner_only']

companies_ownership_by_url = companies_ownership_by_url.sort_values('total_founders_owners', ascending=False)
cofounder_companies = companies_ownership_by_url[companies_ownership_by_url['total_founders_owners'].between(2, 10)]


In [11]:
all_experience['cofound_coown_same_url'] = all_experience['is_founder_or_owner'] & all_experience['company_url'].isin(cofounder_companies['company_url'])


## 2.2 Merge education and experience and create necessary variables

### 2.2.1 Merge data frames

In [12]:
print("Merging experience and education data...")
graduates_with_education_job_level = pd.merge(all_experience, coresignal_member_education, on='member_id',suffixes=('_experience', '_education'))


Merging experience and education data...


### 2.2.2 Create additional variables in merged file

In [13]:

print("Calculating worked_as_engineer and worked_in_sales flags...")
graduates_with_education_job_level['worked_as_engineer'] = graduates_with_education_job_level['title_experience'].str.contains('engineer', case=False, na=False)
graduates_with_education_job_level['worked_in_sales'] = graduates_with_education_job_level['title_experience'].str.contains('sales', case=False, na=False)

# 20_589_319 rows

Calculating worked_as_engineer and worked_in_sales flags...


In [14]:
#Creating cofounder variables with different or same school

## URL-based definitions
cofound_firms = graduates_with_education_job_level.loc[graduates_with_education_job_level['cofound_coown_same_url'] & graduates_with_education_job_level['company_url'].notna(),['company_url', 'unitid']]

number_cofound_schools = pd.DataFrame(cofound_firms.groupby('company_url')['unitid'].nunique(dropna=True))

number_cofound_schools.rename(columns={'unitid': 'num_cofound_schools'}, inplace=True)

graduates_with_education_job_level['cofound_coown_same_url_num_schools'] = graduates_with_education_job_level['company_url'].map(number_cofound_schools['num_cofound_schools']).fillna(0).astype(int)
graduates_with_education_job_level['cofound_coown_same_url_same_school'] = graduates_with_education_job_level['is_founder_or_owner'] & (graduates_with_education_job_level['cofound_coown_same_url_num_schools'] == 1) & graduates_with_education_job_level['company_url'].isin(cofound_firms['company_url'])
graduates_with_education_job_level['cofound_coown_same_url_diff_school'] = graduates_with_education_job_level['is_founder_or_owner'] & (graduates_with_education_job_level['cofound_coown_same_url_num_schools'] > 1) & graduates_with_education_job_level['company_url'].isin(cofound_firms['company_url'])


/tmp/8359077.1.debian.q/ipykernel_259104/3857139298.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  number_cofound_schools = pd.DataFrame(cofound_firms.groupby('company_url')['unitid'].nunique(dropna=True))


## 2.3 Collapse at the level of the graduate instead of the experience

In [15]:
found_own_columns = [col for col in all_experience.columns if 'found' in col.lower() or 'own' in col.lower()]

cols =   ['worked_as_engineer', 'worked_in_sales'] + found_own_columns 
print("cols: ", ", ".join(cols))
    
for col in cols:
    for i in [3,5,10]:
        graduates_with_education_job_level[f'{col}_{i}_years'] = graduates_with_education_job_level[col] & (graduates_with_education_job_level['job_from'] <= (graduates_with_education_job_level['year_to'] + i))

max_cols = {}
for col in cols:
    for i in [3, 5, 10]:
        col_name = f'{col}_{i}_years'
        max_cols[col_name] = 'max'


# Aggregate the data
graduates_person_level = graduates_with_education_job_level.groupby(['member_id', 'year_to', 'year_from', 'title_education', 'subtitle', 'unitid'], dropna=False).agg({    
    **max_cols
    }).reset_index()



cols:  worked_as_engineer, worked_in_sales, is_founder_only, is_owner_only, is_founder_or_owner, is_founder_or_owner_with_url, is_founder_or_owner_inc, is_cofounder_or_coowner_title, is_cofounder_only_title, is_coowner_only_title, franchise_founder_or_owner, franchise_owner, cofound_coown_same_url


In [16]:


# Rename columns for clarity
graduates_person_level.rename(columns={
    'year_from': 'year_start_college',
    'year_to': 'year_end_college',
    'member_id': 'linkedin_member_id',
    'title_education': 'university_title',
    'subtitle': 'university_major_raw'
}, inplace=True)

graduates_person_level.columns


graduates_person_level.sample(20)

,linkedin_member_id,year_end_college,year_start_college,university_title,university_major_raw,unitid,worked_as_engineer_3_years,worked_as_engineer_5_years,worked_as_engineer_10_years,worked_in_sales_3_years,...,is_coowner_only_title_10_years,franchise_founder_or_owner_3_years,franchise_founder_or_owner_5_years,franchise_founder_or_owner_10_years,franchise_owner_3_years,franchise_owner_5_years,franchise_owner_10_years,cofound_coown_same_url_3_years,cofound_coown_same_url_5_years,cofound_coown_same_url_10_years
81570,19933662,2006,2003,buena vista university,"Bachelor of Arts (B.A.), Psychology",153001,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539140,105595774,2011,2007,elon university,"Bachelor of Arts, Strategic Communications",198516,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2107142,355305212,2006,2003,california state university-chico,"Bachelor's Degree, Computer Information Systems",110538,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
2422473,387017189,2007,2003,binghamton university,"Bachelor of Arts (B.A.), Economics",196079,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
743842,136493791,2005,2000,james madison university,"Bachelor of Science (BS), Integrated Science a...",232423,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1989940,339123348,2010,2007,suffolk university,"Bachelor's degree, Journalism",168005,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1025456,179546113,2003,1998,smith college,"Bachelor of Arts (B.A.), Psychology",167835,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2725696,423092888,2002,1999,uw-milwaukee,"B.S., Civil Engineering",240453,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2400611,385161364,2002,1998,west virginia university,"Bachelor of Science, Computer Engineering",238032,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1233438,209991982,1999,1995,shippensburg university of pennsylvania,Bachelor of Science in Education,216010,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## 2.4 Add major to the graduate file

Create the graduate file, one line per graduate.

Keep only those graduates that obtained a bachelors based on the subtitle of the education row (this is a second cleaning)

In [17]:
# Remove rows where university_major_raw contains only generic degree terms without specific major
generic_patterns = [
    r'^bachelor\'?s?\s*degree$',
    r'^b\.?s\.?$',
    r'^b\.?a\.?$', 
    r'^bachelor\'?s?$',
    r'^degree$',
    r'^undergraduate$',
    r'^bachelor of science(\s*\(b\.?s\.?\))?$',
    r'^bachelor of arts(\s*\(b\.?a\.?\))?$',
    r'^bachelors$'
]

# Create a pattern that matches any of the generic patterns (case insensitive)
generic_pattern = '|'.join([f'({pattern})' for pattern in generic_patterns])

# Count rows before filtering
rows_before = len(graduates_person_level)

# Filter out rows with generic degree descriptions
graduates_person_level = graduates_person_level[
    ~graduates_person_level['university_major_raw'].str.lower().str.strip().str.match(generic_pattern, na=False)
]




rows_after = len(graduates_person_level)
print(f"Removed {rows_before - rows_after:,} rows with generic degree descriptions")
print(f"Remaining rows: {rows_after:,}")

Removed 90,892 rows with generic degree descriptions
Remaining rows: 3,004,235


In [18]:
# Clean university_major_raw by removing generic degree prefixes
import re

# Patterns to remove from the beginning of university_major_raw text
patterns_to_remove = [
    r'^bachelor of (applied\s+)?science(\s*\(b\.?s\.?\))?\s*,?\s*',
    r'^bachelor of arts(\s*\(b\.?a\.?\))?\s*,?\s*',
    r'^bachelor\'?s? degree\w?,']

# Combine all patterns
combined_pattern = '|'.join(patterns_to_remove)


# Apply cleaning (case insensitive)
graduates_person_level['university_major_clean'] = graduates_person_level['university_major_raw'].str.replace(
    combined_pattern, '', case=False, regex=True
).str.strip()

graduates_person_level['university_major_clean'] = (
    graduates_person_level['university_major_clean']
    .str.replace(r'\b[Mm]inor\b[ \w]*', '', regex=True)
    .str.strip()
)



# Remove entries that are just short parenthetical notes (e.g., "(BA)", "(BS)")
graduates_person_level = graduates_person_level[~graduates_person_level['university_major_clean'].str.contains(r'^\(.{0:6}\)$')]
print(f"\nAfter cleaning - sample of university_major_raw:")



After cleaning - sample of university_major_raw:


#### 2.2.2 Define the major categorization  keywords

In [19]:
x =  ['asian', 'hispanic', 'african','latin american','gender','feminist','asian american', 'african american','frech','russian','middle eastern','european','caribbean','women\'s','chicano','jewish']
studies_groups_social_science  = [g + ' studies' for g in x]

majors_categories = {
    "Engineering or Computer": {"keywords": ['engineering', 'computer', 'software', 'electronic', 'information systems', 'information technology', 'informatics', 'robotics', 'machine learning', 'artificial intelligence', 'cybersecurity',
                            'architecture', 'urban planning'],
                               "variable_name":"engineering_or_computer"},
    "Natural Science": {"keywords": ['biology', 'biological', 'chemistry', 'physics', 'environmental', 'geology', 'earth', 'astronomy', 'astrophysics', 'meteorology', 'biotechnology', 'biochemistry', 'biotech', 'biochem', 'neuroscience', 'marine', 'oceanography', 'ecology', 'genetics'],
                       "variable_name":"natural_science"},
    "Math": {"keywords": ['math', 'mathematics', 'statistics', 'statistical', 'stats', 'data science', 'analytics'],
             "variable_name":"math"},
    "Education": {"keywords": ['education', 'teacher', 'teaching', 'instructional', 'curriculum', 'pedagogy', 'educational','speech therapy'],
                  "variable_name":"education"},
    "Clinical Work": {"keywords": ['social work', 'pre-med', 'pharmacy', 'nursing', 'health', 'mental', 'therapy', 'clinical', 'counseling'],
                      "variable_name":"clinical_work"},
    "Law / Climinology": {"keywords": ['law', 'legal', 'criminology', 'criminal', 'justice', 'landscape'],
                          "variable_name":"law_climinology"},
    "Economics and Finance": {"keywords": ['economics', 'econ', 'finance', 'financial', 'banking', 'investment', 'econometrics'],
                              "variable_name":"economics_and_finance"},
    "Business (not Economics / Finance)": {"keywords": ['public relations','business', 'management', 'accounting', 'marketing', 'public relations', 'administration', 'advertising', 'human resources', 'operations', 'supply chain', 'organizational behavior'],
                                           "variable_name":"business_not_economics_finance"},
    "Social Science (not Economics)": {"keywords": (['social science', 'history', 'sociology', 'anthropology', 'international relations', 'political science', 'government',
                                                   'policy',  'ethnic', 'cultural', 'religion', 'philosophy','liberal art'] + 
                                studies_groups_social_science)
                                ,"variable_name":"social_science_not_economics"}
                               ,
    "Arts": {"keywords": ['fine art', 'design', 'graphic', 'music', 'theater', 'film', 'cinema', 'photography', 'fashion', 'visual', 'dance', 'performing'],
              "variable_name":"arts"},
    "Communications": {"keywords": ['communication', 'communications', 'media', 'journalism', 'broadcasting'],
                      "variable_name":"communications"},
    "English": {"keywords": ['english', 'literature', 'writing'],
                "variable_name":"english"},
    "Psychology": {"keywords": ['psychology'],
                   "variable_name":"psychology"}
}





In [20]:
subtitle =graduates_person_level['university_major_clean'].dropna().astype(str).sample(100, random_state=42).tolist()[1]

print(f"subtitle: {subtitle}")
subtitle_lower = subtitle.lower()
for major, keywords in majors_categories.items():
    for keyword in keywords:
        if keyword in subtitle_lower:
            print(f"major: {major}")


subtitle: Bachelor of Business Administration, Finance


#### 2.2.3 Run algorithm to categorize majors based on keywords, takes 10 mins or so

In [21]:
from tqdm import tqdm
tqdm.pandas()

# Assign major group to each observation in coresignal_member_education
def assign_major(subtitle, majors_categories):
    if pd.isna(subtitle):
        return None

    subtitle_lower = subtitle.lower()
    for major in majors_categories.keys():
        keywords = majors_categories[major]["keywords"]
        for keyword in keywords:
            if keyword in subtitle_lower:
                return (major, "major_" + majors_categories[major]["variable_name"])
    return ('Other', 'major_other')

print("Applying major assignment to the dataset...")

# Apply the assign_major function and extract results
major_results = graduates_person_level['university_major_clean'].progress_apply(
    lambda x: assign_major(x, majors_categories)
)

# Extract the major categories and variable names
major_categories = [result[0] if result else 'Other' for result in major_results]
major_variables = [result[1] if result else 'major_other' for result in major_results]

# Create university_major_categorized by joining multiple categories with semicolons
graduates_person_level['university_major_categorized'] = [
    '; '.join(sorted(set(cat.split('; ')))) if cat else 'Other' 
    for cat in major_categories
]

# Create dummy variables for each major category
for major_category, details in majors_categories.items():
    var_name = f"major_{details['variable_name']}"
    graduates_person_level[var_name] = [
        1 if var_name in str(major_var) else 0 
        for major_var in major_variables
    ]

# Add major_other dummy variable
graduates_person_level['major_other'] = [
    1 if major_var == 'major_other' else 0 
    for major_var in major_variables
]



Applying major assignment to the dataset...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3004235/3004235 [00:16<00:00, 187112.04it/s]


### 2.4 Data testing and visualizations

In [22]:
print(f'Total number of graduates: {graduates_person_level.shape[0]:,}')

Total number of graduates: 3,004,235


### 2.5 Report summary statistics of file

In [23]:
# Print summary statistics for all major categories
print("Summary Statistics for Major Categories:")
print("=" * 50)

# Calculate percentages for each major category
total_graduates = len(graduates_person_level)
print(f"Total graduates: {total_graduates:,}")
print()

# Print statistics for each major category dummy variable
major_dummy_cols = [col for col in graduates_person_level.columns if col.startswith('major_')]
for col in major_dummy_cols:
    count = graduates_person_level[col].sum()
    percentage = (count / total_graduates) * 100
    print(f"{col:<35}: {count:>8,} ({percentage:>5.1f}%)")

print()
print("University Major Categorized Distribution:")
print("-" * 45)
major_dist = graduates_person_level['university_major_categorized'].value_counts()
for category, count in major_dist.items():
    percentage = (count / total_graduates) * 100
    print(f"{category:<35}: {count:>8,} ({percentage:>5.1f}%)")


Summary Statistics for Major Categories:
Total graduates: 3,004,235

major_engineering_or_computer      :  341,492 ( 11.4%)
major_natural_science              :  243,127 (  8.1%)
major_math                         :   37,656 (  1.3%)
major_education                    :  142,493 (  4.7%)
major_clinical_work                :  134,298 (  4.5%)
major_law_climinology              :   83,627 (  2.8%)
major_economics_and_finance        :  180,592 (  6.0%)
major_business_not_economics_finance:  672,701 ( 22.4%)
major_social_science_not_economics :  298,981 ( 10.0%)
major_arts                         :  172,703 (  5.7%)
major_communications               :  166,455 (  5.5%)
major_english                      :   93,017 (  3.1%)
major_psychology                   :  148,997 (  5.0%)
major_other                        :  288,096 (  9.6%)

University Major Categorized Distribution:
---------------------------------------------
Business (not Economics / Finance) :  672,701 ( 22.4%)
Engineering or 

In [24]:
graduates_person_level.sample(5000)[['university_major_raw','university_major_clean','university_major_categorized']].to_csv('graduates_person_level_majors_sample5000.csv', index=False)

# 3 Store files for Stata analysis

In [25]:
# Create variable labels dictionary for Stata export
variable_labels = {
    # Core Identification Variables
    'linkedin_member_id': 'Unique LinkedIn member identifier',
    'unitid': 'University identification code from IPEDS database',
    
    # Educational Background Variables
    'university_title': 'Official name of the university',
    'university_major_raw': 'Raw major/degree description as reported',
    'university_major_clean': 'Cleaned major description with degree prefixes removed',
    'university_major_categorized': 'Major classified into standardized categories',
    'year_start_college': 'Year the individual started college',
    'year_end_college': 'Year the individual graduated from college',
    
    # Major Category Dummy Variables
    'major_engineering_or_computer': 'Engineering, computer science, software, IT, robotics, AI',
    'major_natural_science': 'Biology, chemistry, physics, environmental science, geology',
    'major_math': 'Mathematics, statistics, data science, analytics',
    'major_education': 'Education, teaching, instructional design, curriculum',
    'major_clinical_work': 'Social work, pre-med, pharmacy, nursing, health, therapy',
    'major_law_climinology': 'Law, legal studies, criminology, criminal justice',
    'major_economics_and_finance': 'Economics, finance, banking, investment, econometrics',
    'major_business_not_economics_finance': 'Business, management, accounting, marketing, PR, admin',
    'major_social_science_not_economics': 'History, sociology, anthropology, political science',
    'major_arts': 'Fine arts, design, music, theater, film, photography',
    'major_communications': 'Communications, media, journalism, broadcasting',
    'major_english': 'English, literature, writing',
    'major_psychology': 'Psychology',
    'major_other': 'All other majors not categorized above',
    
    # 3-Year Entrepreneurship Variables
    'is_founder_only_3_years': 'Founded company within 3 years (title contains founder)',
    'is_owner_only_3_years': 'Owned company within 3 years (title contains owner)',
    'is_founder_or_owner_3_years': 'Founded or owned company within 3 years',
    'is_cofounder_or_coowner_title_3_years': 'Co-founded or co-owned company within 3 years',
    'is_cofounder_only_title_3_years': 'Co-founded company within 3 years',
    'is_coowner_only_title_3_years': 'Co-owned company within 3 years',
    'is_founder_or_owner_with_url_3_years': 'Founded/owned company with LinkedIn URL within 3 years',
    'is_founder_or_owner_inc_3_years': 'Founded/owned incorporated company within 3 years',
    'cofound_coown_same_url_3_years': 'Co-founded/co-owned company (same URL) within 3 years',
    'cofound_coown_same_url_num_schools_3_years': 'Number different universities among co-founders within 3 years',
    'cofound_coown_same_url_same_school_3_years': 'Co-founded with same university graduate within 3 years',
    'cofound_coown_same_url_diff_school_3_years': 'Co-founded with different university graduate within 3 years',
    
    # 5-Year Entrepreneurship Variables
    'is_founder_only_5_years': 'Founded company within 5 years (title contains founder)',
    'is_owner_only_5_years': 'Owned company within 5 years (title contains owner)',
    'is_founder_or_owner_5_years': 'Founded or owned company within 5 years',
    'is_cofounder_or_coowner_title_5_years': 'Co-founded or co-owned company within 5 years',
    'is_cofounder_only_title_5_years': 'Co-founded company within 5 years',
    'is_coowner_only_title_5_years': 'Co-owned company within 5 years',
    'is_founder_or_owner_with_url_5_years': 'Founded/owned company with LinkedIn URL within 5 years',
    'is_founder_or_owner_inc_5_years': 'Founded/owned incorporated company within 5 years',
    'cofound_coown_same_url_5_years': 'Co-founded/co-owned company (same URL) within 5 years',
    'cofound_coown_same_url_num_schools_5_years': 'Number different universities among co-founders within 5 years',
    'cofound_coown_same_url_same_school_5_years': 'Co-founded with same university graduate within 5 years',
    'cofound_coown_same_url_diff_school_5_years': 'Co-founded with different university graduate within 5 years',
    
    # 10-Year Entrepreneurship Variables
    'is_founder_only_10_years': 'Founded company within 10 years (title contains founder)',
    'is_owner_only_10_years': 'Owned company within 10 years (title contains owner)',
    'is_founder_or_owner_10_years': 'Founded or owned company within 10 years',
    'is_cofounder_or_coowner_title_10_years': 'Co-founded or co-owned company within 10 years',
    'is_cofounder_only_title_10_years': 'Co-founded company within 10 years',
    'is_coowner_only_title_10_years': 'Co-owned company within 10 years',
    'is_founder_or_owner_with_url_10_years': 'Founded/owned company with LinkedIn URL within 10 years',
    'is_founder_or_owner_inc_10_years': 'Founded/owned incorporated company within 10 years',
    'cofound_coown_same_url_10_years': 'Co-founded/co-owned company (same URL) within 10 years',
    'cofound_coown_same_url_num_schools_10_years': 'Number different universities among co-founders within 10 years',
    'cofound_coown_same_url_same_school_10_years': 'Co-founded with same university graduate within 10 years',
    'cofound_coown_same_url_diff_school_10_years': 'Co-founded with different university graduate within 10 years',
    
    # Career Experience Variables
    'worked_as_engineer_3_years': 'Worked in engineering role within 3 years',
    'worked_as_engineer_5_years': 'Worked in engineering role within 5 years',
    'worked_as_engineer_10_years': 'Worked in engineering role within 10 years',
    'worked_in_sales_3_years': 'Worked in sales role within 3 years',
    'worked_in_sales_5_years': 'Worked in sales role within 5 years',
    'worked_in_sales_10_years': 'Worked in sales role within 10 years'
}

# Add labels as metadata to the DataFrame
graduates_person_level.attrs['variable_labels'] = variable_labels

# Display how many variables have labels
labeled_vars = [col for col in graduates_person_level.columns if col in variable_labels]
unlabeled_vars = [col for col in graduates_person_level.columns if col not in variable_labels]

print(f"Variables with labels: {len(labeled_vars)}")
print(f"Variables without labels: {len(unlabeled_vars)}")

if unlabeled_vars:
    print(f"\nUnlabeled variables: {unlabeled_vars}")
    
print(f"\nTotal variables in dataset: {len(graduates_person_level.columns)}")
print(f"Sample of labeled variables:")
for i, (var, label) in enumerate(list(variable_labels.items())[:5]):
    print(f"  {var}: {label}")

Variables with labels: 55
Variables without labels: 6

Unlabeled variables: ['franchise_founder_or_owner_3_years', 'franchise_founder_or_owner_5_years', 'franchise_founder_or_owner_10_years', 'franchise_owner_3_years', 'franchise_owner_5_years', 'franchise_owner_10_years']

Total variables in dataset: 61
Sample of labeled variables:
  linkedin_member_id: Unique LinkedIn member identifier
  unitid: University identification code from IPEDS database
  university_title: Official name of the university
  university_major_raw: Raw major/degree description as reported
  university_major_clean: Cleaned major description with degree prefixes removed


In [26]:

#Scratch
#graduates_person_level = pd.read_stata("graduates_person_level_091182025.dta")


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(graduates_person_level.year_start_college.value_counts().sort_index())

# Add: subtitle to a cleaned major
# Founded within years: finer from 1 to 10 years, every year -- make count per year, not cumulative.

year_start_college
1900         9
1901         1
1903         1
1904         1
1915         1
1921         1
1939         1
1943         1
1950         1
1952         1
1954         2
1959         3
1960         4
1961         5
1962         2
1963         4
1964         6
1965        12
1966        14
1967        22
1968        26
1969        25
1970        29
1971        28
1972        39
1973        46
1974        45
1975        64
1976        65
1977        83
1978       147
1979       186
1980       154
1981       119
1982       154
1983       170
1984       218
1985       268
1986       410
1987       550
1988      1274
1989      2116
1990      1286
1991      1699
1992      5036
1993     14948
1994     21198
1995     28376
1996     37721
1997     60801
1998    142153
1999    174289
2000    226156
2001    233685
2002    255738
2003    276177
2004    307590
2005    330420
2006    359803
2007    342681
2008    139111
2009     31716
2010      6880
2011       463
Name: count, dtype: I

In [44]:
# -----------------------------------------------------------
# Each row is one person
# University that they went to --> well matched into a single ID using Runjing's data.

# Start and end year and montyh of bachelor's degree
     # would be cool to show collaboration across groups. 

# Do they start a firm within 3, 5, or 10 years

# -----------------------------------------------------------
# Separate dataset of founding events: person id, firm id, founding date, location of firm. 

print("Storing graduates_person_level", flush=True)
#graduates_person_level = graduates_person_level[~graduates_person_level.linkedin_member_id.duplicated() ]
graduates_person_level.to_stata("graduates_person_level_10082025.dta",  version=118)




Storing graduates_person_level


MemoryError: Unable to allocate 2.87 MiB for an array with shape (3004235,) and data type bool

In [28]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(graduates_person_level.describe(include='all').transpose())



,count,unique,top,freq,mean,std,min,25%,50%,75%,max
linkedin_member_id,3004235.0,NaN,NaN,NaN,259184674.139576,141171821.969228,4093.0,141205596.0,255695512.0,375659176.0,631792515.0
year_end_college,3004235.0,<NA>,<NA>,<NA>,2006.85434,3.205673,1997.0,2005.0,2007.0,2010.0,2011.0
year_start_college,3004235.0,<NA>,<NA>,<NA>,2003.048618,3.523131,1900.0,2001.0,2004.0,2006.0,2011.0
university_title,3004128,5597,michigan state university,21747,NaN,NaN,NaN,NaN,NaN,NaN,NaN
university_major_raw,3004235,808503,"Bachelor of Arts (B.A.), Psychology",26194,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unitid,3004235.0,NaN,NaN,NaN,178317.191608,48477.453819,100654.0,142285.0,174491.0,213400.0,482413.0
worked_as_engineer_3_years,3004235,2,False,2798245,NaN,NaN,NaN,NaN,NaN,NaN,NaN
worked_as_engineer_5_years,3004235,2,False,2770776,NaN,NaN,NaN,NaN,NaN,NaN,NaN
worked_as_engineer_10_years,3004235,2,False,2717065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
worked_in_sales_3_years,3004235,2,False,2727966,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Summary Statistics for graduates_person_level Dataset

Comprehensive overview of the dataset structure and variable distributions.

In [29]:
# Basic dataset information
print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {graduates_person_level.shape}")
print(f"Number of observations: {graduates_person_level.shape[0]:,}")
print(f"Number of variables: {graduates_person_level.shape[1]:,}")
print(f"Memory usage: {graduates_person_level.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nData types:")
print(graduates_person_level.dtypes.value_counts())
print("\n" + "="*60)

=== DATASET OVERVIEW ===
Dataset shape: (3004235, 61)
Number of observations: 3,004,235
Number of variables: 61
Memory usage: 1615.21 MB

Data types:
boolean    39
int64      15
object      4
Int64       2
int32       1
Name: count, dtype: int64



In [30]:
# Missing values analysis
print("=== MISSING VALUES ANALYSIS ===")
missing_summary = graduates_person_level.isnull().sum()
missing_percentage = (missing_summary / len(graduates_person_level)) * 100

missing_df = pd.DataFrame({
    'Variable': missing_summary.index,
    'Missing_Count': missing_summary.values,
    'Missing_Percentage': missing_percentage.values
}).sort_values('Missing_Percentage', ascending=False)

print(f"Variables with missing values: {(missing_df['Missing_Count'] > 0).sum()}")
print(f"Variables with no missing values: {(missing_df['Missing_Count'] == 0).sum()}")
print("\nTop 20 variables with highest missing values:")
print(missing_df.head(20).to_string(index=False))
print("\n" + "="*60)

=== MISSING VALUES ANALYSIS ===
Variables with missing values: 1
Variables with no missing values: 60

Top 20 variables with highest missing values:
                   Variable  Missing_Count  Missing_Percentage
           university_title            107            0.003562
         linkedin_member_id              0            0.000000
           year_end_college              0            0.000000
         year_start_college              0            0.000000
       university_major_raw              0            0.000000
                     unitid              0            0.000000
 worked_as_engineer_3_years              0            0.000000
 worked_as_engineer_5_years              0            0.000000
worked_as_engineer_10_years              0            0.000000
    worked_in_sales_3_years              0            0.000000
    worked_in_sales_5_years              0            0.000000
   worked_in_sales_10_years              0            0.000000
    is_founder_only_3_years     

In [33]:
import numpy as np
# Numeric variables summary statistics
print("=== NUMERIC VARIABLES SUMMARY ===")
numeric_cols = graduates_person_level.select_dtypes(include=[np.number]).columns
print(f"Number of numeric variables: {len(numeric_cols)}")

if len(numeric_cols) > 0:
    numeric_summary = graduates_person_level[numeric_cols].describe()
    print("\nDetailed summary statistics for numeric variables:")
    print(numeric_summary.round(3))
    
    # Additional statistics for numeric variables
    print("\nAdditional numeric statistics:")
    additional_stats = pd.DataFrame({
        'Variable': numeric_cols,
        'Median': graduates_person_level[numeric_cols].median(),
        'Mode': graduates_person_level[numeric_cols].mode().iloc[0] if len(graduates_person_level[numeric_cols].mode()) > 0 else np.nan,
        'Variance': graduates_person_level[numeric_cols].var(),
        'Skewness': graduates_person_level[numeric_cols].skew(),
        'Kurtosis': graduates_person_level[numeric_cols].kurtosis()
    }).round(3)
    print(additional_stats.to_string(index=False))

print("\n" + "="*60)

=== NUMERIC VARIABLES SUMMARY ===
Number of numeric variables: 18

Detailed summary statistics for numeric variables:
       linkedin_member_id  year_end_college  year_start_college       unitid  \
count        3.004235e+06         3004235.0           3004235.0  3004235.000   
mean         2.591847e+08          2006.854            2003.049   178317.192   
std          1.411718e+08             3.206               3.523    48477.454   
min          4.093000e+03            1997.0              1900.0   100654.000   
25%          1.412056e+08            2005.0              2001.0   142285.000   
50%          2.556955e+08            2007.0              2004.0   174491.000   
75%          3.756592e+08            2010.0              2006.0   213400.000   
max          6.317925e+08            2011.0              2011.0   482413.000   

       major_engineering_or_computer  major_natural_science   major_math  \
count                    3004235.000            3004235.000  3004235.000   
mean     

In [34]:
# Categorical variables summary
print("=== CATEGORICAL VARIABLES SUMMARY ===")
categorical_cols = graduates_person_level.select_dtypes(include=['object', 'category']).columns
print(f"Number of categorical variables: {len(categorical_cols)}")

if len(categorical_cols) > 0:
    print("\nCategorical variables overview:")
    cat_summary = []
    for col in categorical_cols:
        unique_count = graduates_person_level[col].nunique()
        most_frequent = graduates_person_level[col].mode().iloc[0] if len(graduates_person_level[col].mode()) > 0 else 'N/A'
        most_frequent_count = graduates_person_level[col].value_counts().iloc[0] if len(graduates_person_level[col].value_counts()) > 0 else 0
        
        cat_summary.append({
            'Variable': col,
            'Unique_Values': unique_count,
            'Most_Frequent': str(most_frequent)[:50] + ('...' if len(str(most_frequent)) > 50 else ''),
            'Most_Frequent_Count': most_frequent_count,
            'Most_Frequent_Pct': (most_frequent_count / len(graduates_person_level)) * 100
        })
    
    cat_df = pd.DataFrame(cat_summary)
    print(cat_df.round(2).to_string(index=False))

print("\n" + "="*60)

=== CATEGORICAL VARIABLES SUMMARY ===
Number of categorical variables: 4

Categorical variables overview:
                    Variable  Unique_Values                       Most_Frequent  Most_Frequent_Count  Most_Frequent_Pct
            university_title           5597           michigan state university                21747               0.72
        university_major_raw         808503 Bachelor of Arts (B.A.), Psychology                26194               0.87
      university_major_clean         690915                          Psychology                91376               3.04
university_major_categorized             14  Business (not Economics / Finance)               672701              22.39



In [35]:
# Boolean/Binary variables summary
print("=== BOOLEAN/BINARY VARIABLES SUMMARY ===")
bool_cols = []
binary_cols = []

# Identify boolean and binary columns
for col in graduates_person_level.columns:
    unique_vals = graduates_person_level[col].dropna().unique()
    if graduates_person_level[col].dtype == 'bool':
        bool_cols.append(col)
    elif len(unique_vals) == 2 and set(unique_vals).issubset({0, 1, True, False, 'True', 'False', 'yes', 'no', 'Yes', 'No'}):
        binary_cols.append(col)

print(f"Boolean variables: {len(bool_cols)}")
print(f"Binary variables: {len(binary_cols)}")

all_binary = bool_cols + binary_cols
if len(all_binary) > 0:
    print(f"\nSummary of {len(all_binary)} boolean/binary variables:")
    binary_summary = []
    for col in all_binary:
        value_counts = graduates_person_level[col].value_counts()
        if len(value_counts) >= 2:
            binary_summary.append({
                'Variable': col,
                'True/1_Count': value_counts.iloc[0] if value_counts.index[0] in [1, True, 'True', 'yes', 'Yes'] else value_counts.iloc[1],
                'False/0_Count': value_counts.iloc[1] if value_counts.index[0] in [1, True, 'True', 'yes', 'Yes'] else value_counts.iloc[0],
                'True_Percentage': (value_counts.iloc[0] / len(graduates_person_level)) * 100 if value_counts.index[0] in [1, True, 'True', 'yes', 'Yes'] else (value_counts.iloc[1] / len(graduates_person_level)) * 100
            })
    
    if binary_summary:
        binary_df = pd.DataFrame(binary_summary)
        print(binary_df.round(2).to_string(index=False))

print("\n" + "="*60)

=== BOOLEAN/BINARY VARIABLES SUMMARY ===
Boolean variables: 0
Binary variables: 53

Summary of 53 boolean/binary variables:
                              Variable  True/1_Count  False/0_Count  True_Percentage
            worked_as_engineer_3_years        205990        2798245             6.86
            worked_as_engineer_5_years        233459        2770776             7.77
           worked_as_engineer_10_years        287170        2717065             9.56
               worked_in_sales_3_years        276269        2727966             9.20
               worked_in_sales_5_years        327011        2677224            10.89
              worked_in_sales_10_years        410518        2593717            13.66
               is_founder_only_3_years         29078        2975157             0.97
               is_founder_only_5_years         43881        2960354             1.46
              is_founder_only_10_years         83613        2920622             2.78
                 is_owner_

In [36]:
# Key entrepreneurship and education variables summary
print("=== KEY VARIABLES ANALYSIS ===")

# Education variables
education_vars = [col for col in graduates_person_level.columns if 'education' in col.lower() or 'degree' in col.lower() or 'school' in col.lower()]
print(f"Education-related variables: {len(education_vars)}")
if education_vars:
    print("Education variables:", education_vars[:10], "..." if len(education_vars) > 10 else "")

# Entrepreneurship variables
entrepreneur_vars = [col for col in graduates_person_level.columns if any(keyword in col.lower() for keyword in ['found', 'entrepreneur', 'startup', 'cofounder', 'owner'])]
print(f"\nEntrepreneurship-related variables: {len(entrepreneur_vars)}")
if entrepreneur_vars:
    print("Entrepreneurship variables:", entrepreneur_vars[:10], "..." if len(entrepreneur_vars) > 10 else "")

# Time-windowed variables
time_vars = [col for col in graduates_person_level.columns if any(time in col for time in ['_3_years', '_5_years', '_10_years'])]
print(f"\nTime-windowed variables: {len(time_vars)}")
if time_vars:
    print("Time-windowed variables:", time_vars[:10], "..." if len(time_vars) > 10 else "")

# Major category variables
major_vars = [col for col in graduates_person_level.columns if 'major_' in col.lower()]
print(f"\nMajor category variables: {len(major_vars)}")
if major_vars:
    print("Major variables:", major_vars[:10], "..." if len(major_vars) > 10 else "")

print("\n" + "="*60)

=== KEY VARIABLES ANALYSIS ===
Education-related variables: 1
Education variables: ['major_education'] 

Entrepreneurship-related variables: 33
Entrepreneurship variables: ['is_founder_only_3_years', 'is_founder_only_5_years', 'is_founder_only_10_years', 'is_owner_only_3_years', 'is_owner_only_5_years', 'is_owner_only_10_years', 'is_founder_or_owner_3_years', 'is_founder_or_owner_5_years', 'is_founder_or_owner_10_years', 'is_founder_or_owner_with_url_3_years'] ...

Time-windowed variables: 39
Time-windowed variables: ['worked_as_engineer_3_years', 'worked_as_engineer_5_years', 'worked_as_engineer_10_years', 'worked_in_sales_3_years', 'worked_in_sales_5_years', 'worked_in_sales_10_years', 'is_founder_only_3_years', 'is_founder_only_5_years', 'is_founder_only_10_years', 'is_owner_only_3_years'] ...

Major category variables: 17
Major variables: ['university_major_raw', 'university_major_clean', 'university_major_categorized', 'major_engineering_or_computer', 'major_natural_science', 'maj

In [37]:
# Sample data preview for key variables
print("=== SAMPLE DATA PREVIEW ===")

# Show a sample of key variables
key_vars = ['person_id', 'university_name', 'graduation_year'] + \
           [col for col in graduates_person_level.columns if 'major_' in col][:5] + \
           [col for col in graduates_person_level.columns if 'found' in col.lower()][:5]

# Filter to existing columns
key_vars = [col for col in key_vars if col in graduates_person_level.columns]

if key_vars:
    print(f"Sample of {len(key_vars)} key variables for first 10 observations:")
    print(graduates_person_level[key_vars].head(10).to_string(max_cols=None))
else:
    print("Key variables not found, showing first 5 columns:")
    print(graduates_person_level.iloc[:10, :5].to_string())

print("\n" + "="*60)

=== SAMPLE DATA PREVIEW ===
Sample of 10 key variables for first 10 observations:
                                                                         university_major_raw                                                     university_major_clean        university_major_categorized  major_engineering_or_computer  major_natural_science  is_founder_only_3_years  is_founder_only_5_years  is_founder_only_10_years  is_founder_or_owner_3_years  is_founder_or_owner_5_years
0                                                        Bachelor of Arts (B.A.), Advertising                                                                Advertising  Business (not Economics / Finance)                              0                      0                    False                    False                     False                        False                        False
1                                      Bachelor's Degree, Registered Nursing/Registered Nurse                                       

In [39]:
# Data quality assessment
print("=== DATA QUALITY ASSESSMENT ===")

# Check for duplicates
duplicate_count = graduates_person_level.duplicated().sum()
print(f"Duplicate rows: {duplicate_count:,} ({(duplicate_count/len(graduates_person_level)*100):.2f}%)")

# Check for completely empty rows
empty_rows = graduates_person_level.isnull().all(axis=1).sum()
print(f"Completely empty rows: {empty_rows:,}")

# Check for rows with mostly missing data (>80% missing)
missing_threshold = 0.8
mostly_missing = (graduates_person_level.isnull().sum(axis=1) / graduates_person_level.shape[1]) > missing_threshold
print(f"Rows with >{missing_threshold*100}% missing data: {mostly_missing.sum():,}")

# Check data consistency for key ID variables
if 'person_id' in graduates_person_level.columns:
    unique_ids = graduates_person_level['person_id'].nunique()
    total_rows = len(graduates_person_level)
    print(f"Unique person IDs: {unique_ids:,} (vs {total_rows:,} total rows)")
    if unique_ids != total_rows:
        print(f"  → Multiple records per person: {total_rows - unique_ids:,} duplicate person records")

print(f"\nOverall data completeness: {((graduates_person_level.notna().sum().sum()) / (graduates_person_level.shape[0] * graduates_person_level.shape[1]) * 100):.2f}%")

print("\n" + "="*60)
print("SUMMARY STATISTICS COMPLETED")
print("="*60)

=== DATA QUALITY ASSESSMENT ===
Duplicate rows: 0 (0.00%)
Completely empty rows: 0
Rows with >80.0% missing data: 0

Overall data completeness: 100.00%

SUMMARY STATISTICS COMPLETED


## Testing internally of the graduate file


In [41]:
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np

import matplotlib.pyplot as plt

# Load the graduates_person_level data if not already loaded
#graduates_person_level = pd.read_pickle('graduates_person_level_AnalysisFile.pkl')

# Prepare the data for regression
# Create dummy variables for college-year fixed effects
graduates_person_level['college_year'] = graduates_person_level['unitid'].astype(str) + '_' + graduates_person_level['year_end_college'].astype(str)

# Get dummies for college-year fixed effects
college_year_dummies = pd.get_dummies(graduates_person_level['college_year'], prefix='college_year')

# Prepare X variables (graduation year + college-year fixed effects)
X = pd.concat([
    graduates_person_level[['year_end_college']],
    college_year_dummies
], axis=1)

# Use founded_within_10_years as dependent variable
y = graduates_person_level['founded_within_10_years']

# Create interaction terms for major categories with graduation year
major_dummy_vars = [col for col in graduates_person_level.columns if col.startswith('major_')]

# Create interaction terms (graduation year * major dummy)
for major_var in major_dummy_vars:
    graduates_person_level[f'{major_var}_x_year'] = graduates_person_level[major_var] * graduates_person_level['year_end_college']

# Add interaction terms to X variables
interaction_cols = [col for col in graduates_person_level.columns if col.endswith('_x_year')]
X = pd.concat([
    graduates_person_level[['year_end_college']],
    graduates_person_level[major_dummy_vars],
    graduates_person_level[interaction_cols],
    college_year_dummies
], axis=1)

# Remove rows with missing values
mask = ~(X.isna().any(axis=1) | y.isna())
X_clean = X[mask]
y_clean = y[mask]

print(f"Running regression with {len(X_clean):,} observations")
print(f"Number of features: {X_clean.shape[1]}")

# Fit the regression
reg = LinearRegression()
reg.fit(X_clean, y_clean)

# Get coefficient for year_end_college
year_coef = reg.coef_[0]
print(f"Coefficient for year_end_college: {year_coef:.6f}")

# Plot the coefficient
plt.figure(figsize=(10, 6))
plt.bar(['Year of Graduation'], [year_coef], color='steelblue', alpha=0.7)
plt.title('Effect of Graduation Year on Founding a Company Within 10 Years')
plt.ylabel('Coefficient')
plt.xlabel('Variable')
plt.grid(axis='y', alpha=0.3)

# Add value label on bar
plt.text(0, year_coef + (abs(year_coef) * 0.1), f'{year_coef:.6f}', 
         ha='center', va='bottom' if year_coef > 0 else 'top')

plt.tight_layout()
plt.show()

# Print regression summary
print(f"\nRegression Results:")
print(f"R-squared: {reg.score(X_clean, y_clean):.6f}")
print(f"Number of observations: {len(X_clean):,}")
print(f"Mean of dependent variable: {y_clean.mean():.6f}")

MemoryError: Unable to allocate 91.8 GiB for an array with shape (32819, 3004235) and data type bool

In [ ]:
graduates_person_level.columns

In [ ]:
import pandas as pd
from linearmodels.iv import AbsorbingLS

graduates_person_level['college_year'] = graduates_person_level['unitid'].astype(str) + '_' + graduates_person_level['year_end_college'].astype(str)

# Make FEs categorical to save memory
df = graduates_person_level.assign(
    fe1=lambda d: d.unitid.astype('category'),    
    fe_x=lambda d: d.year_end_college.astype('int16')    
).dropna(subset=['founded_within_10_years','year_end_college','fe1','fe_x'])

# Prepare dependent and independent variables
dependent = df['founded_within_10_years']
exog = pd.get_dummies(graduates_person_level['year_end_college'], prefix='xx')
absorb = df[['unitid']]   # Fixed effects variable



# Create and fit the model
mod = AbsorbingLS(dependent, exog, absorb=absorb)
res = mod.fit(
    cov_type='clustered',
    clusters=df[['fe1']]     # one- or multi-way clustering
)
print(res.summary)


In [ ]:

universities_adopted_facebook.fb_date.value_counts()
#graduates_person_level.columns
#universities_adopted_facebook.columns

In [ ]:

# Read the company data CSV file efficiently
print("Reading coresignal_company.csv file...")
print("Note: This is a large file (11GB), loading may take several minutes...")

coresignal_company = pd.read_csv('coresignal_company.csv')
print(f"Company data loaded: {len(coresignal_company):,} rows, {len(coresignal_company.columns)} columns")
print("Columns in company data:", coresignal_company.columns.tolist())

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

# First, let's examine the data we're working with
print("Founder events shape:", founder_events.shape)
print("Company data shape:", coresignal_company.shape)

# Check the company_url column in founder_events
print("\nCompany URL info in founder_events:")
print(f"Total founder events: {len(founder_events):,}")
print(f"Non-null company URLs: {founder_events['company_url'].notna().sum():,}")
print(f"Missing company URLs: {founder_events['company_url'].isna().sum():,}")

# Check if there's a URL column in coresignal_company
print("\nColumns in coresignal_company that might contain URLs:")
url_columns = [col for col in coresignal_company.columns if 'url' in col.lower() or 'link' in col.lower()]
print(url_columns)

# Also check for company name columns
name_columns = [col for col in coresignal_company.columns if 'name' in col.lower() or 'company' in col.lower()]
print("\nColumns in coresignal_company that might contain company names:")
print(name_columns)

# to dos and different changes after 10/06


GET THE DATA TO RUNJING 

- Create a better definition of co-founder if we can (being part of two companies by URL)
    - Then add co-founding across different colleges. 
    - 
- Move the  franchisees into their own definition.



# Data Dictionary for graduates_person_level Dataset

This data dictionary describes all variables in the `graduates_person_level` dataset, which contains one row per graduate with their educational background and entrepreneurial outcomes.

## Core Identification Variables
- **linkedin_member_id**: Unique LinkedIn member identifier (renamed from member_id)
- **unitid**: University identification code from IPEDS database

## Educational Background Variables
- **university_title**: Official name of the university (renamed from title_education)
- **university_major_raw**: Raw major/degree description as reported (renamed from subtitle)
- **university_major_clean**: Cleaned version of major description with degree prefixes removed
- **university_major_categorized**: Major classified into standardized categories
- **year_start_college**: Year the individual started college (renamed from year_from)
- **year_end_college**: Year the individual graduated from college (renamed from year_to)

## Major Category Dummy Variables (1 if major falls in category, 0 otherwise)
- **major_engineering_or_computer**: Engineering, computer science, software, IT, robotics, AI, cybersecurity, architecture
- **major_natural_science**: Biology, chemistry, physics, environmental science, geology, astronomy, biotechnology, neuroscience
- **major_math**: Mathematics, statistics, data science, analytics
- **major_education**: Education, teaching, instructional design, curriculum, pedagogy
- **major_clinical_work**: Social work, pre-med, pharmacy, nursing, health, therapy, clinical work, counseling
- **major_law_climinology**: Law, legal studies, criminology, criminal justice
- **major_economics_and_finance**: Economics, finance, banking, investment, econometrics
- **major_business_not_economics_finance**: Business, management, accounting, marketing, PR, administration, advertising, HR
- **major_social_science_not_economics**: History, sociology, anthropology, political science, international relations, cultural studies
- **major_arts**: Fine arts, design, music, theater, film, photography, fashion, visual arts, dance
- **major_communications**: Communications, media, journalism, broadcasting
- **major_english**: English, literature, writing
- **major_psychology**: Psychology
- **major_other**: All other majors not categorized above

## Entrepreneurship Outcome Variables (Within X Years of Graduation)

### 3-Year Variables
- **is_founder_only_3_years**: Founded a company within 3 years (title contains "founder")
- **is_owner_only_3_years**: Owned a company within 3 years (title contains "owner")
- **is_founder_or_owner_3_years**: Either founded or owned a company within 3 years
- **is_cofounder_or_coowner_title_3_years**: Co-founded or co-owned a company within 3 years (title-based)
- **is_cofounder_only_title_3_years**: Co-founded a company within 3 years (title contains "co-founder")
- **is_coowner_only_title_3_years**: Co-owned a company within 3 years (title contains "co-owner")
- **is_founder_or_owner_with_url_3_years**: Founded/owned company with LinkedIn URL within 3 years
- **is_founder_or_owner_inc_3_years**: Founded/owned incorporated company within 3 years
- **cofound_coown_same_url_3_years**: Co-founded/co-owned company (same URL definition) within 3 years
- **cofound_coown_same_url_num_schools_3_years**: Number of different universities among co-founders within 3 years
- **cofound_coown_same_url_same_school_3_years**: Co-founded with someone from same university within 3 years
- **cofound_coown_same_url_diff_school_3_years**: Co-founded with someone from different university within 3 years

### 5-Year Variables
- **is_founder_only_5_years**: Founded a company within 5 years
- **is_owner_only_5_years**: Owned a company within 5 years
- **is_founder_or_owner_5_years**: Either founded or owned a company within 5 years
- **is_cofounder_or_coowner_title_5_years**: Co-founded or co-owned a company within 5 years
- **is_cofounder_only_title_5_years**: Co-founded a company within 5 years
- **is_coowner_only_title_5_years**: Co-owned a company within 5 years
- **is_founder_or_owner_with_url_5_years**: Founded/owned company with LinkedIn URL within 5 years
- **is_founder_or_owner_inc_5_years**: Founded/owned incorporated company within 5 years
- **cofound_coown_same_url_5_years**: Co-founded/co-owned company (same URL definition) within 5 years
- **cofound_coown_same_url_num_schools_5_years**: Number of different universities among co-founders within 5 years
- **cofound_coown_same_url_same_school_5_years**: Co-founded with someone from same university within 5 years
- **cofound_coown_same_url_diff_school_5_years**: Co-founded with someone from different university within 5 years

### 10-Year Variables
- **is_founder_only_10_years**: Founded a company within 10 years
- **is_owner_only_10_years**: Owned a company within 10 years
- **is_founder_or_owner_10_years**: Either founded or owned a company within 10 years
- **is_cofounder_or_coowner_title_10_years**: Co-founded or co-owned a company within 10 years
- **is_cofounder_only_title_10_years**: Co-founded a company within 10 years
- **is_coowner_only_title_10_years**: Co-owned a company within 10 years
- **is_founder_or_owner_with_url_10_years**: Founded/owned company with LinkedIn URL within 10 years
- **is_founder_or_owner_inc_10_years**: Founded/owned incorporated company within 10 years
- **cofound_coown_same_url_10_years**: Co-founded/co-owned company (same URL definition) within 10 years
- **cofound_coown_same_url_num_schools_10_years**: Number of different universities among co-founders within 10 years
- **cofound_coown_same_url_same_school_10_years**: Co-founded with someone from same university within 10 years
- **cofound_coown_same_url_diff_school_10_years**: Co-founded with someone from different university within 10 years

## Career Experience Variables (Within X Years of Graduation)
- **worked_as_engineer_3_years**: Worked in engineering role within 3 years
- **worked_as_engineer_5_years**: Worked in engineering role within 5 years
- **worked_as_engineer_10_years**: Worked in engineering role within 10 years
- **worked_in_sales_3_years**: Worked in sales role within 3 years
- **worked_in_sales_5_years**: Worked in sales role within 5 years
- **worked_in_sales_10_years**: Worked in sales role within 10 years

## Data Notes
1. **Sample**: Individuals who graduated with bachelor's degrees between 1995-2012
2. **Entrepreneurship Definition**: Based on job titles containing "founder" or "owner" keywords
3. **Data Cleaning**: Excluded non-ownership roles like "product owner", "program owner", titles with 4+ words
4. **Co-founding**: Defined as multiple founders/owners at same company (by LinkedIn URL)
5. **Time Windows**: All outcome variables measured within 3, 5, or 10 years post-graduation
6. **Incorporated Companies**: Companies with names containing "inc", "corp", "corporation", "co", "incorporated"
7. **Major Categories**: Based on keyword matching in degree descriptions

## Data Structure
- **Unit of Analysis**: One row per graduate
- **Time Frame**: Graduates from 1995-2012, outcomes tracked through ~2020s
- **Sample Size**: Approximately 1.5+ million graduates
- **Data Source**: LinkedIn profiles via Coresignal, matched to IPEDS university data

In [ ]:
# Create variable labels dictionary for Stata export
variable_labels = {
    # Core Identification Variables
    'linkedin_member_id': 'Unique LinkedIn member identifier',
    'unitid': 'University identification code from IPEDS database',
    
    # Educational Background Variables
    'university_title': 'Official name of the university',
    'university_major_raw': 'Raw major/degree description as reported',
    'university_major_clean': 'Cleaned major description with degree prefixes removed',
    'university_major_categorized': 'Major classified into standardized categories',
    'year_start_college': 'Year the individual started college',
    'year_end_college': 'Year the individual graduated from college',
    
    # Major Category Dummy Variables
    'major_engineering_or_computer': 'Engineering, computer science, software, IT, robotics, AI',
    'major_natural_science': 'Biology, chemistry, physics, environmental science, geology',
    'major_math': 'Mathematics, statistics, data science, analytics',
    'major_education': 'Education, teaching, instructional design, curriculum',
    'major_clinical_work': 'Social work, pre-med, pharmacy, nursing, health, therapy',
    'major_law_climinology': 'Law, legal studies, criminology, criminal justice',
    'major_economics_and_finance': 'Economics, finance, banking, investment, econometrics',
    'major_business_not_economics_finance': 'Business, management, accounting, marketing, PR, admin',
    'major_social_science_not_economics': 'History, sociology, anthropology, political science',
    'major_arts': 'Fine arts, design, music, theater, film, photography',
    'major_communications': 'Communications, media, journalism, broadcasting',
    'major_english': 'English, literature, writing',
    'major_psychology': 'Psychology',
    'major_other': 'All other majors not categorized above',
    
    # 3-Year Entrepreneurship Variables
    'is_founder_only_3_years': 'Founded company within 3 years (title contains founder)',
    'is_owner_only_3_years': 'Owned company within 3 years (title contains owner)',
    'is_founder_or_owner_3_years': 'Founded or owned company within 3 years',
    'is_cofounder_or_coowner_title_3_years': 'Co-founded or co-owned company within 3 years',
    'is_cofounder_only_title_3_years': 'Co-founded company within 3 years',
    'is_coowner_only_title_3_years': 'Co-owned company within 3 years',
    'is_founder_or_owner_with_url_3_years': 'Founded/owned company with LinkedIn URL within 3 years',
    'is_founder_or_owner_inc_3_years': 'Founded/owned incorporated company within 3 years',
    'cofound_coown_same_url_3_years': 'Co-founded/co-owned company (same URL) within 3 years',
    'cofound_coown_same_url_num_schools_3_years': 'Number different universities among co-founders within 3 years',
    'cofound_coown_same_url_same_school_3_years': 'Co-founded with same university graduate within 3 years',
    'cofound_coown_same_url_diff_school_3_years': 'Co-founded with different university graduate within 3 years',
    
    # 5-Year Entrepreneurship Variables
    'is_founder_only_5_years': 'Founded company within 5 years (title contains founder)',
    'is_owner_only_5_years': 'Owned company within 5 years (title contains owner)',
    'is_founder_or_owner_5_years': 'Founded or owned company within 5 years',
    'is_cofounder_or_coowner_title_5_years': 'Co-founded or co-owned company within 5 years',
    'is_cofounder_only_title_5_years': 'Co-founded company within 5 years',
    'is_coowner_only_title_5_years': 'Co-owned company within 5 years',
    'is_founder_or_owner_with_url_5_years': 'Founded/owned company with LinkedIn URL within 5 years',
    'is_founder_or_owner_inc_5_years': 'Founded/owned incorporated company within 5 years',
    'cofound_coown_same_url_5_years': 'Co-founded/co-owned company (same URL) within 5 years',
    'cofound_coown_same_url_num_schools_5_years': 'Number different universities among co-founders within 5 years',
    'cofound_coown_same_url_same_school_5_years': 'Co-founded with same university graduate within 5 years',
    'cofound_coown_same_url_diff_school_5_years': 'Co-founded with different university graduate within 5 years',
    
    # 10-Year Entrepreneurship Variables
    'is_founder_only_10_years': 'Founded company within 10 years (title contains founder)',
    'is_owner_only_10_years': 'Owned company within 10 years (title contains owner)',
    'is_founder_or_owner_10_years': 'Founded or owned company within 10 years',
    'is_cofounder_or_coowner_title_10_years': 'Co-founded or co-owned company within 10 years',
    'is_cofounder_only_title_10_years': 'Co-founded company within 10 years',
    'is_coowner_only_title_10_years': 'Co-owned company within 10 years',
    'is_founder_or_owner_with_url_10_years': 'Founded/owned company with LinkedIn URL within 10 years',
    'is_founder_or_owner_inc_10_years': 'Founded/owned incorporated company within 10 years',
    'cofound_coown_same_url_10_years': 'Co-founded/co-owned company (same URL) within 10 years',
    'cofound_coown_same_url_num_schools_10_years': 'Number different universities among co-founders within 10 years',
    'cofound_coown_same_url_same_school_10_years': 'Co-founded with same university graduate within 10 years',
    'cofound_coown_same_url_diff_school_10_years': 'Co-founded with different university graduate within 10 years',
    
    # Career Experience Variables
    'worked_as_engineer_3_years': 'Worked in engineering role within 3 years',
    'worked_as_engineer_5_years': 'Worked in engineering role within 5 years',
    'worked_as_engineer_10_years': 'Worked in engineering role within 10 years',
    'worked_in_sales_3_years': 'Worked in sales role within 3 years',
    'worked_in_sales_5_years': 'Worked in sales role within 5 years',
    'worked_in_sales_10_years': 'Worked in sales role within 10 years'
}

print(f"Created variable labels for {len(variable_labels)} variables")

In [50]:
# Check which variables exist in graduates_person_level and apply labels
if 'graduates_person_level' in locals():
    # Check which labeled variables exist in the dataset
    existing_vars = set(graduates_person_level.columns)
    labeled_vars = set(variable_labels.keys())
    
    # Variables that exist and have labels
    vars_with_labels = existing_vars & labeled_vars
    # Variables that exist but don't have labels
    vars_without_labels = existing_vars - labeled_vars
    # Variables that have labels but don't exist
    labels_without_vars = labeled_vars - existing_vars
    
    print("Variable Label Matching Summary:")
    print("=" * 40)
    print(f"Total variables in dataset: {len(existing_vars)}")
    print(f"Variables with labels: {len(vars_with_labels)}")
    print(f"Variables without labels: {len(vars_without_labels)}")
    print(f"Labels without matching variables: {len(labels_without_vars)}")
    
    if len(vars_without_labels) > 0 and len(vars_without_labels) <= 10:
        print(f"\nVariables without labels:")
        for var in sorted(vars_without_labels):
            print(f"  {var}")
    elif len(vars_without_labels) > 10:
        print(f"\nFirst 10 variables without labels:")
        for var in sorted(list(vars_without_labels)[:10]):
            print(f"  {var}")
        print(f"  ... and {len(vars_without_labels)-10} more")
    
    # Create final labels dictionary with only existing variables
    final_labels = {var: variable_labels[var] for var in vars_with_labels}
    
    # Add labels as metadata to the DataFrame
    graduates_person_level.attrs['variable_labels'] = final_labels
    
    print(f"\nApplied {len(final_labels)} variable labels to graduates_person_level DataFrame")
    
    # Export to Stata with variable labels
    print("\nExporting to Stata with variable labels...")
    graduates_person_level.to_stata(
        "graduates_person_level_with_labels_10082025.dta",
        variable_labels=final_labels,
        version=118,
        write_index=False
    )
    
    print(f"Successfully exported {len(graduates_person_level):,} observations to Stata file")
    print(f"File: graduates_person_level_with_labels_10072025.dta")
    print("Variable labels will be visible in Stata using 'describe' or 'codebook' commands")
    
else:
    print("graduates_person_level DataFrame not found in memory")
    print("You may need to run the earlier cells to create the dataset first")

Variable Label Matching Summary:
Total variables in dataset: 62
Variables with labels: 55
Variables without labels: 7
Labels without matching variables: 9

Variables without labels:
  college_year
  franchise_founder_or_owner_10_years
  franchise_founder_or_owner_3_years
  franchise_founder_or_owner_5_years
  franchise_owner_10_years
  franchise_owner_3_years
  franchise_owner_5_years

Applied 55 variable labels to graduates_person_level DataFrame

Exporting to Stata with variable labels...


/tmp/8359077.1.debian.q/ipykernel_259104/3365217772.py:41: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    is_founder_or_owner_with_url_3_years   ->   is_founder_or_owner_with_url_3_y
    is_founder_or_owner_with_url_5_years   ->   is_founder_or_owner_with_url_5_y
    is_founder_or_owner_with_url_10_years   ->   is_founder_or_owner_with_url_10_
    is_cofounder_or_coowner_title_3_years   ->   is_cofounder_or_coowner_title_3_
    is_cofounder_or_coowner_title_5_years   ->   is_cofounder_or_coowner_title_5_
    is_cofounder_or_coowner_title_10_years   ->   is_cofounder_or_coowner_title_10
    franchise_founder_or_owner_3_years   ->   franchise_founder_or_owner_3_yea
    franchise_founder_or_owner_5_years   ->   franchise_founder_or_owner_5_yea
    franchise_founder_or_owner_10_years   ->   franchise_founder_or_owner_10_ye
    major_business_not_economics_finance   ->   major_business_not_economics_fin
    ma

Successfully exported 3,004,235 observations to Stata file
File: graduates_person_level_with_labels_10072025.dta
Variable labels will be visible in Stata using 'describe' or 'codebook' commands


In [ ]:
# Convert all binary variables to int type for cleaner summary statistics
if 'graduates_person_level' in locals():
    # Identify binary variables (variables with only 0, 1, and NaN values)
    binary_vars = []
    
    for col in graduates_person_level.columns:
        if graduates_person_level[col].dtype in ['bool', 'float64', 'int64']:
            unique_vals = set(graduates_person_level[col].dropna().unique())
            # Check if values are only 0 and 1 (or subset thereof)
            if unique_vals.issubset({0, 1, 0.0, 1.0, True, False}):
                binary_vars.append(col)
    
    print(f"Converting {len(binary_vars)} binary variables to int type:")
    print("Binary variables found:")
    for var in binary_vars[:10]:  # Show first 10
        print(f"  {var}")
    if len(binary_vars) > 10:
        print(f"  ... and {len(binary_vars) - 10} more")
    
    # Convert binary variables to int, handling NaN values
    graduates_person_level_clean = graduates_person_level.copy()
    for col in binary_vars:
        # Convert to int, keeping NaN as NaN
        graduates_person_level_clean[col] = graduates_person_level_clean[col].astype('Int64')
    
    print(f"\nConversion complete. Data types updated for {len(binary_vars)} variables.")
    
else:
    print("graduates_person_level DataFrame not found in memory")

In [ ]:
# Generate comprehensive summary statistics with integer binary variables
if 'graduates_person_level_clean' in locals():
    
    print("COMPREHENSIVE SUMMARY STATISTICS - GRADUATES_PERSON_LEVEL")
    print("=" * 80)
    
    # Basic dataset info
    print(f"Dataset Shape: {graduates_person_level_clean.shape[0]:,} rows × {graduates_person_level_clean.shape[1]} columns")
    print(f"Memory Usage: {graduates_person_level_clean.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    print()
    
    # 1. CORE IDENTIFICATION VARIABLES
    print("1. CORE IDENTIFICATION VARIABLES")
    print("-" * 40)
    id_vars = ['linkedin_member_id', 'unitid']
    for var in id_vars:
        if var in graduates_person_level_clean.columns:
            series = graduates_person_level_clean[var]
            print(f"{var}:")
            print(f"  Non-null: {series.notna().sum():,} ({series.notna().mean()*100:.1f}%)")
            print(f"  Unique values: {series.nunique():,}")
    print()
    
    # 2. EDUCATIONAL BACKGROUND VARIABLES
    print("2. EDUCATIONAL BACKGROUND VARIABLES")
    print("-" * 40)
    edu_vars = ['year_start_college', 'year_end_college', 'university_major_categorized']
    for var in edu_vars:
        if var in graduates_person_level_clean.columns:
            series = graduates_person_level_clean[var]
            if series.dtype in ['int64', 'Int64', 'float64']:
                print(f"{var}: Mean={series.mean():.1f}, Min={series.min()}, Max={series.max()}, Missing={series.isna().sum():,}")
            else:
                print(f"{var}: {series.nunique():,} unique values, Missing={series.isna().sum():,}")
    print()
    
    # 3. MAJOR CATEGORIES (showing as percentages)
    print("3. MAJOR CATEGORY DISTRIBUTION")
    print("-" * 40)
    major_vars = [col for col in graduates_person_level_clean.columns if col.startswith('major_')]
    total_grads = len(graduates_person_level_clean)
    
    for var in sorted(major_vars):
        count = graduates_person_level_clean[var].sum()
        pct = (count / total_grads) * 100
        print(f"{var:<35}: {count:>8,} ({pct:>5.1f}%)")
    print()
    
    # 4. ENTREPRENEURSHIP OUTCOMES (3, 5, 10 years)
    print("4. ENTREPRENEURSHIP OUTCOMES BY TIME HORIZON")
    print("-" * 40)
    
    for years in [3, 5, 10]:
        print(f"\n{years}-Year Outcomes:")
        outcome_vars = [col for col in graduates_person_level_clean.columns if col.endswith(f'_{years}_years')]
        
        for var in sorted(outcome_vars)[:8]:  # Show first 8 to avoid clutter
            if graduates_person_level_clean[var].dtype in ['int64', 'Int64', 'bool']:
                count = graduates_person_level_clean[var].sum()
                pct = (count / total_grads) * 100
                print(f"  {var:<40}: {count:>6,} ({pct:>4.1f}%)")
    
    print()
    
    # 5. DESCRIPTIVE STATISTICS FOR KEY VARIABLES
    print("5. DESCRIPTIVE STATISTICS FOR CONTINUOUS VARIABLES")
    print("-" * 40)
    continuous_vars = ['year_start_college', 'year_end_college']
    continuous_vars += [col for col in graduates_person_level_clean.columns if 'num_schools' in col]
    
    desc_stats = graduates_person_level_clean[continuous_vars].describe()
    print(desc_stats.round(2))
    print()
    
    # 6. MISSING DATA SUMMARY
    print("6. MISSING DATA SUMMARY")
    print("-" * 40)
    missing_summary = graduates_person_level_clean.isnull().sum()
    missing_pct = (missing_summary / len(graduates_person_level_clean)) * 100
    
    vars_with_missing = missing_summary[missing_summary > 0].sort_values(ascending=False)
    if len(vars_with_missing) > 0:
        print("Variables with missing data:")
        for var, count in vars_with_missing.head(10).items():
            pct = missing_pct[var]
            print(f"  {var:<40}: {count:>6,} ({pct:>5.1f}%)")
        if len(vars_with_missing) > 10:
            print(f"  ... and {len(vars_with_missing) - 10} more variables with missing data")
    else:
        print("No missing data found!")
    
    print("\n" + "=" * 80)
    print("Summary statistics generation complete!")
    
else:
    print("graduates_person_level_clean not found. Run the previous cell first to convert data types.")